In [2]:
import nibabel as nib
import numpy as np
import torchio as tio
import os

c:\Users\wijflo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tuple_product(*args):
    product = 1
    for element in args:
        product *= element
    return product

In [3]:
def resize_images_to_reference (source_path, destination_path, ref_image_path=None, image_is_label=False, ref_spacing=None, interpolation_method="linear", modification_string="", inclusion_string="", save_ref=False):
    files = os.listdir(source_path)

    # Select files to process.
    nifti_files = [file for file in files if (file.endswith('.nii.gz')) & (inclusion_string in file)]
    nii_img_shapes = []

    # First get a list of image shapes.
    for file in nifti_files:
        file_path = os.path.join(source_path, file)
        nii_img = nib.load(file_path)
        nii_img_shapes.append(nii_img.shape)
    
    nifti_files_to_process = nifti_files.copy()

    # No need to add a reference image if a spacing is given
    if ref_spacing != None:
        pass
    # If no reference image is used, dynamically choose a reference among selected files by choosing the biggest image.
    elif ref_image_path is None:
        index_of_max_product = max(range(len(nii_img_shapes)), key=lambda i: tuple_product(nii_img_shapes[i]))
        del nifti_files_to_process[index_of_max_product]
        
        reference_filename = nifti_files[index_of_max_product]
        reference_image = tio.ScalarImage(os.path.join(source_path, reference_filename))
        
    else:
        reference_filename = os.path.basename(ref_image_path)
        reference_image = tio.ScalarImage(ref_image_path)
    
    if ref_spacing is None:
        # Define the reference image name, adding the modification string if necessary.
        split_name = reference_filename.split(".")
        if modification_string != "":
            split_name[0] = split_name[0] + "_" + modification_string
        new_reference_img_name = ".".join(split_name)
    
        if save_ref:
            reference_image.save(os.path.join(destination_path, new_reference_img_name))

        print("Processed reference image ", reference_filename)

    # Use tio.Resample to resize all images to match the shape of the reference image.
    for file in nifti_files_to_process:
        file_path = os.path.join(source_path, file)

        if image_is_label:
            image_to_resize = tio.LabelMap(file_path)
        else:
            image_to_resize = tio.ScalarImage(file_path)
        if ref_spacing is None:
            normalized_image = tio.Resample(target=reference_image)(image_to_resize)
        else:
            normalized_image = tio.Resample(target=ref_spacing, image_interpolation=interpolation_method)(image_to_resize)
        
        split_name = file.split(".")
        if modification_string != "":
            split_name[0] = split_name[0] + "_" + modification_string
        new_img_name = ".".join(split_name)

        normalized_image.save(os.path.join(destination_path, new_img_name))
        
        print("Processed image ", file)

In [3]:
def resize_images_to_moving_reference (source_path, target_path, ref_folder_path, image_is_label=False, interpolation_method="linear", modification_string="", inclusion_string=""):
    # Resize a folder of images according to a reference of another folder of images. There must be a 1-to-1 correspondence between the resized and reference folder.

    # Select files to process.
    nifti_files_source = [file for file in os.listdir(source_path) if (file.endswith('.nii.gz')) & (inclusion_string in file)]
    nifti_files_ref = [file for file in os.listdir(ref_folder_path) if (file.endswith('.nii.gz')) & (inclusion_string in file)]

    if len(nifti_files_source) != len(nifti_files_ref):
        raise SystemExit("Number of images to resize and their reference is different.")
    
    for source_file, ref_file in zip(nifti_files_source, nifti_files_ref):
        source_split_name = source_file.split("_")
        source_file_path = os.path.join(source_path, source_file)
        ref_file_path = os.path.join(ref_folder_path, ref_file)

        if image_is_label:
            image_to_resize = tio.LabelMap(source_file_path)
            ref_image = tio.ScalarImage(ref_file_path)
            normalized_image = tio.Resample(target=ref_image)(image_to_resize)
        else:
            image_to_resize = tio.ScalarImage(source_file_path)
            ref_image = tio.ScalarImage(ref_file_path)
            normalized_image = tio.Resample(target=ref_image, image_interpolation=interpolation_method)(image_to_resize)

        split_name = source_file.split(".")
        if modification_string != "":
            split_name[0] = split_name[0] + "_" + modification_string
        
        new_img_name = ".".join(split_name)
        normalized_image.save(os.path.join(target_path, new_img_name))
        print("Processed image ", source_file) 

In [4]:
resize_images_to_moving_reference ("G:\\Data_Booster\\data_ETIS_781\\IntensityNormalized_Images\\Raw\\TOF3D", "G:\\Data_Booster\\data_ETIS_781\\Resized_Images\\SWI_asReference_ImagebyImage\\TOF3D", "G:\\Data_Booster\\data_ETIS_781\\SkullStripping\\Raw\\SWI_Skullstripped", image_is_label=False, interpolation_method="lanczos", modification_string="Resized", inclusion_string="")

Processed image  2018-104_01-10087-D0MR_8_TOF_3D_NEW_wo_ARTEFACTS_fl3d1r_t50_Normalized.nii.gz
Processed image  2018-104_01-10113-D0MR_6_3D_TOF_LARGE__Normalized.nii.gz
Processed image  2018-104_01-10114-D0MR_6_3D_TOF_LARGE__Normalized.nii.gz
Processed image  2018-104_01-10116-D0MR_8_TOF_3D_WILLIS_FIN_Normalized.nii.gz
Processed image  2018-104_01-10117-D0MR_6_3D_TOF_LARGE_Normalized.nii.gz
Processed image  2018-104_01-10118-D0MR_8_TOF_3D_RAPIDE_Normalized.nii.gz
Processed image  2018-104_01-10120-D0MR_501_3D_TOF_HS_Normalized.nii.gz
Processed image  2018-104_01-10124-D0MR_5_3D_TOF_HS_Normalized.nii.gz
Processed image  2018-104_01-10126-D0MR_8_TOF_3D_Normalized.nii.gz
Processed image  2018-104_01-10128-D0MR_5_3D_TOF_HS_Normalized.nii.gz
Processed image  2018-104_01-10129-D0MR_7_TOF_3D_NEW_wo_ARTEFACTS_Normalized.nii.gz
Processed image  2018-104_01-10136-D0MR_7_TOF_3D_NEW_wo_ARTEFACTS_Normalized.nii.gz
Processed image  2018-104_01-10151-D0MR_6_3D_TOF_LARGE_Normalized.nii.gz
Processed im

In [8]:
resize_images_to_reference("G:\\Data_Booster\\data_ETIS_781\\DIFF", "G:\\Data_Booster\\data_ETIS_781\\Resized_Images\\DIFF", ref_image_path="G:\\Data_Booster\\data_ETIS_781\\Resized_Images\SWI\\2018-104_01-10087-D0MR_12_SWI_Resized_Images.nii.gz", modification_string="Resized")

Processed reference image  2018-104_01-10087-D0MR_12_SWI_Resized_Images.nii.gz
Processed image  2018-104_01-10087-D0MR_106_AX_DIFF_RESOLVE_TRACEW.nii.gz
Processed image  2018-104_01-10113-D0MR_3_Ax_Diffusion_b1000.nii.gz
Processed image  2018-104_01-10114-D0MR_3_Ax_Diffusion_b1000.nii.gz
Processed image  2018-104_01-10116-D0MR_5_AX_DIFFUSION_SMS_B2000_TRACEW.nii.gz
Processed image  2018-104_01-10117-D0MR_3_Ax_Diffusion_b1000.nii.gz
Processed image  2018-104_01-10118-D0MR_141_AX_DIFF_RESOLVE_TRACEW.nii.gz
Processed image  2018-104_01-10120-D0MR_55_AX_DIFF_RESOLVE_TRACEW.nii.gz
Processed image  2018-104_01-10124-D0MR_3_Ax_DWI.nii.gz
Processed image  2018-104_01-10126-D0MR_105_AX_DIFF_RESOLVE_TRACEW.nii.gz
Processed image  2018-104_01-10128-D0MR_3_Ax_DWI.nii.gz
Processed image  2018-104_01-10129-D0MR_105_AX_DIFF_RESOLVE_TRACEW.nii.gz
Processed image  2018-104_01-10136-D0MR_105_AX_DIFF_RESOLVE_TRACEW.nii.gz
Processed image  2018-104_01-10151-D0MR_3_Ax_Diffusion_b1000.nii.gz
Processed image

In [6]:
resize_images_to_reference("D:\\data_ETIS_781\\Resized\\MASK", "D:\\data_ETIS_781\\Downsampling_1p5_2p2\\MASK_Downsampled", interpolation_method="lanczos", ref_spacing=(1.5,1.5,2.2), modification_string="Downsampled")

Processed image  2018-104_01-10087-D0MR_S12_202312051300_Resized_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10113-D0MR_S9_202312051300_Resized_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10114-D0MR_S7_202312051300_Resized_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10116-D0MR_S7_202312051300_Resized_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10117-D0MR_S7_202312051300_Resized_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10118-D0MR_S12_202312051300_Resized_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10120-D0MR_S8_202312051300_Resized_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10124-D0MR_S6_202312051300_Resized_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10126-D0MR_S12_202312051300_Resized_Tours-THROMBMICS-Clot-Segmentation.nii.gz
Processed image  2018-104_01-10128-D0MR_S7_